In [50]:
import numpy as np
import pandas as pd

In [51]:
from sklearn.datasets import load_iris

data = load_iris(as_frame=True)
dataset = data.frame
targets = data.target_names
dataset.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [52]:
targets

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [53]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    np.array(dataset[dataset.columns.drop(['target'])]),
    np.array(dataset['target']),
    test_size = 0.2,
    random_state=42
)

In [54]:
from src.layers import BNormalizer

normalizer = BNormalizer(4)
normalizer.fit(x_train)

x_train = normalizer.predict(x_train)

In [55]:
def one_hot_encode(labels, target_names):
    one_hot_targets = []
    for label in labels:
        target = [0]*len(target_names)
        target[label] = 1
        one_hot_targets.append(target)
    return np.array(one_hot_targets)

In [56]:
y_train, y_test = one_hot_encode(y_train, targets), one_hot_encode(y_test, targets)

In [57]:
from src.model import BModel
from src.layers import BDenseLayer
from src.loss_function import categorical_crossentropy

model = BModel([
    BDenseLayer(16, 32),
    BDenseLayer(32, 32),
    BDenseLayer(32, 32),
    BDenseLayer(32, 32),
    BDenseLayer(32, 32),
    BDenseLayer(32, 32),
    BDenseLayer(32, 3)
], categorical_crossentropy)

In [59]:
hist = model.fit(x_train, y_train, batch_size=32, meta_parameter_size=10, epochs=5)

Epoch 1/5:


2it [00:00, 15.32it/s]

2.014762043869794
3.022143015804689
2.5184525298372415


4it [00:00, 16.31it/s]


3.3579366731163205
Epoch 2/5:


...

2.014762043869794
Epoch 5/5:


2it [00:00, 17.36it/s]

1.5110715579023468
3.022143015804689
3.022143015804689


4it [00:00, 18.50it/s]

3.3579366731163205
